# Prepare SQuAD_tiny Dataset for Assignment 2

This code prepare SQuAD_tiny from the SQuAD dataset. 

# 0. Import libraries

In [1]:
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import logging as transformers_logging

In [2]:
# Set seed for reproducibility
torch.manual_seed(42)

# 1. Load and preprocess SQuAD dataset

In [ ]:
# 1. Load and preprocess SQuAD dataset
dataset = load_dataset("squad")

README.md: 0.00B [00:00, ?B/s]

c:\conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\23179\.cache\huggingface\hub\datasets--squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install t

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
# Take subsets to avoid overload
train_dataset = dataset["train"].select(range(10000,11000))
val_dataset = dataset["validation"].select(range(3000,3100))
test_dataset = dataset["validation"].select(range(3100, 3200))  # No official SQuAD test set

In [5]:
print("Size of training set:", len(train_dataset))
print("Size of validation set:", len(val_dataset))
print("Size of testing set:", len(test_dataset))

Size of training set: 1000
Size of validation set: 100
Size of testing set: 100


In [7]:
MODEL_NAME = "t5-small"
#MODEL_NAME = "t5-base"
MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [8]:
def encode_question_and_context(question, context):
    return f"question: {question}  context: {context}"

# Obtains the context, question and answer from a given sample.
def extract_sample_parts(sample):
    context = sample["context"]
    question = sample["question"]
    answer = sample["answers"]['text'][0]
    question_with_context = encode_question_and_context(question, context)
    return (question_with_context, question, answer)

# Encodes the sample, returning token IDs.
def preprocess(sample):
    # Extract data from sample.
    question_with_context, question, answer = extract_sample_parts(sample)

    # Using truncation causes the tokenizer to emit a warning for every sample.
    # This will generate a significant amount of messages, and likely crash
    # your browser tab. We temporarily disable log messages to work around this.
    # See https://github.com/huggingface/transformers/issues/14285
    old_level = transformers_logging.get_verbosity()
    transformers_logging.set_verbosity_error()
    
    # Generate tokens for the input.
    # We include both the context and the question (first two parameters).
    input_tokens = tokenizer(question_with_context, question, padding="max_length",
                             truncation=True, max_length=MAX_INPUT_LENGTH)

    # Generate tokens for the expected answer. There is no need to include the 
    output_tokens = tokenizer(answer, padding="max_length", truncation=True,
                              max_length=MAX_OUTPUT_LENGTH)

    # Restore old logging level, see above.
    transformers_logging.set_verbosity(old_level)

    # The output of the tokenizer is a map containing {input_ids, attention_mask}.
    # For trianing, we need to add the labels (answer/output tokens) to the map.
    input_tokens["labels"] = np.array(output_tokens["input_ids"])

    return input_tokens

In [9]:
# Preprocess the datasets
training_set_enc = train_dataset.map(preprocess, batched=False)
validation_set_enc = val_dataset.map(preprocess, batched=False)
testing_set_enc = test_dataset.map(preprocess, batched=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
# Prepare 20 data points for qualitative analysis
q_data = test_dataset.select(range(20))
q_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 20
})

In [11]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    logging as transformers_logging
)
from rouge_score import rouge_scorer

In [12]:
torch.manual_seed(42)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [20]:
print(dataset['train'][0]['question'])
print(dataset['train'][0]['answers']['text'][0])
print(dataset['train'][0]['context'][:200]) 

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Saint Bernadette Soubirous
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper sta


In [14]:
print("Number of training examples:",len(dataset['train']))
dataset['train'][0]

Number of training examples: 87599


{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [17]:
#Data Filtering

filtered_dataset = dataset.filter(lambda x:x["answers"]['text'][0].upper())!= ("CANNOTANSWER")
print("Size of training set after removing unanswerable questions:",len(filtered_dataset['train']))
print("Size of validation set after removing unanswerable questions:",
    len(filtered_dataset['validation']))

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

TypeError: 'bool' object is not subscriptable

In [22]:
filtered_dataset = dataset.filter(lambda x: x["answers"]["text"][0].upper() != "CANNOTANSWER")

print("Size of training set after removing unanswerable questions:", len(filtered_dataset["train"]))
print("Size of validation set after removing unanswerable questions:", len(filtered_dataset["validation"]))

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

Size of training set after removing unanswerable questions: 87599
Size of validation set after removing unanswerable questions: 10570


In [23]:
#creating testing set

training_set = filtered_dataset['train']
validation_set = filtered_dataset['validation'].select(range(500))
testing_set = filtered_dataset['validation'].select(range(500,len(filtered_dataset['validation'])))

print("Size of training set:", len(training_set))
print("Size of validation set:", len(validation_set))
print("Size of testing set:", len(testing_set))

Size of training set: 87599
Size of validation set: 500
Size of testing set: 10070


In [ ]:
#Loading the Tokenizer

#preprocessing required for feeding data into the T5 model 
# converting the text into a series of "tokens"

MODEL_NAME = "t5-small"
MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
#hand-typed

'''#Preprocessing Data

def encode_question_and_context(question,context):
    return f"question:{question} context: {context}"

#obtain the context,question and answer from a given sample.
def extract_sample_parts(sample):
    context = sample["context"]
    question = sample["questions"][0]
    answer = sample["orig_answers"]['texts'][0]
    question_with_context = encode_question_and_context(question,context)
    return (question_with_context, question, answer)

#encode the sample, returning the token IDs.

def preprocess(sample):
    question_with_context, question, answer = extract_sample_parts(sample)
    old_level = transformers_logging.get_verbosity()
    transformers_logging.set_verbosity_error()
    input_tokens = tokenizer(answer,padding="max length",truncation=True,
                             max_length=MAX_OUTPUT_LENGTH)
    transformers_logging.set_verbosity(old_level)

    input_tokens["labels"] = np.array(output_tokens["input_ids"])

    return input_tokens'''

In [25]:
def encode_question_and_context(question,context):
    return f"question:{question} context: {context}"

def extract_sample_parts(sample):
    context = sample["context"]
    question = sample["question"]
    answer = sample["answers"]["text"][0]
    question_with_context = encode_question_and_context(question, context)
    return (question_with_context, question, answer)

def preprocess(sample):
    question_with_context, question, answer = extract_sample_parts(sample)
    
    old_level = transformers_logging.get_verbosity()
    transformers_logging.set_verbosity_error()

    input_tokens = tokenizer(
        question_with_context,
        padding="max_length",
        truncation=True,
        max_length=MAX_INPUT_LENGTH
    )

    output_tokens = tokenizer(
        answer,
        padding="max_length",
        truncation=True,
        max_length=MAX_OUTPUT_LENGTH
    )

    transformers_logging.set_verbosity(old_level)

    input_tokens["labels"] = np.array(output_tokens["input_ids"])

    return input_tokens


In [26]:
#encoded
#apply the preprocess() function to each sample of the dataset
training_set_enc = training_set.map(preprocess, batched=False)
validation_set_enc = validation_set.map(preprocess)
testing_set_enc = testing_set.map(preprocess, batched=False)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/10070 [00:00<?, ? examples/s]

In [27]:
#for check

example = training_set_enc[0]
print(example.keys())

print("\nDecoded input:")
print(tokenizer.decode(example["input_ids"], skip_special_tokens=True))

print("\nDecoded label:")
print(tokenizer.decode(example["labels"], skip_special_tokens=True))


dict_keys(['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'])

Decoded input:
question:To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Decoded label:
Saint Bernadette Soubirous


In [28]:
#Loading the Model

try:
    del model
except NameError:
    pass

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [29]:
#Fine-Tuning the Model

columns = ["input_ids", "attention_mask", "labels"]
training_set_enc.set_format(type="torch", columns=columns)
validation_set_enc.set_format(type="torch", columns=columns)
testing_set_enc.set_format(type="torch", columns=columns)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results", 
    num_train_epochs=3, 
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8, 
    eval_strategy="epoch", 
    save_strategy="epoch", 
    learning_rate=3e-4, 
    weight_decay=0.01, 
    save_total_limit=2, 
    logging_dir="./logs", 
    logging_steps=10 )

In [31]:
# switch the model to training mode, enabling dropout layers.
model.train()
trainer = Trainer(
model=model,
args=training_args,
train_dataset=training_set_enc,
eval_dataset=validation_set_enc,
processing_class=tokenizer,
data_collator=DataCollatorForSeq2Seq(tokenizer)
)
trainer.train()

c:\conda\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\conda\Lib\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Saving the model
trainer.save_model("my_model")
# Reloading the model
model = T5ForConditionalGeneration.from_pretrained("./my_model")
tokenizer = T5Tokenizer.from_pretrained("./my_model")
trainer.save_model("t5_pretrained")

In [ ]:
#model evaluation